In [1]:
import pandas as pd

In [84]:
excel_schedule=pd.read_excel('examples/расписание 19.Б01-э осенний семестр .xlsx')
CURRENT_YEAR = 2021

In [592]:
df = excel_schedule.copy()
df =  df.drop(df.columns[0], axis=1)
df = df.drop([0,1], axis=0)
df = df.reset_index()
df =  df.drop(df.columns[0], axis=1)

In [593]:
column_name = ['Time', 'Date','SubjectName', 'Place', 'Teacher']
df = df.set_axis((column_name), axis=1, inplace=False)
df = df.drop([0], axis=0)
# df.head()

In [594]:
from typing import List


def to_datetime(date: str, year: int):
    return dt.datetime.strptime(date + f'.{year}', "%d.%m.%Y").date()


def find_two_week_dates(dates) -> List:
    output_data = []
    date_start = dates[0]
    date_end = dates[0]
    repeat_time = 1
    repeat_type = 'biweekly'
    for date in dates:
        if date_end + dt.timedelta(days=14) == date:
            date_end = date
            repeat_time += 1
        else:
            if date_end == date:
                continue
            output_data.append((date_start, date_end, repeat_time, repeat_type))
            date_start, date_end, repeat_time = date, date, 1
    output_data.append((date_start, date_end, repeat_time, repeat_type))
    return output_data


def find_dates(x: str, year: int) -> List:
    if x.find('\n') != -1:
        dates = list(map(lambda date: to_datetime(date, year), x.split('\n')))
        return find_two_week_dates(dates)
    elif x[-1] == ')':
        dates, repeat_time = x.split('(')
        date_start, date_end = dates.strip().split('–')
        repeat_time = int(repeat_time[:-1])
    else:
        date_start = x
        date_end = x
        repeat_time = 1
    date_start = dt.datetime.strptime(date_start + f'.{year}', "%d.%m.%Y").date()
    date_end = dt.datetime.strptime(date_end + f'.{year}', "%d.%m.%Y").date()
    repeat_type='weekly'
    return [(date_start, date_end, repeat_time, repeat_type)]


def parse_date(x: str, year: int) -> List:
    if x.find('–') != -1 and x.find('\n') != -1:
        dates = []
        for date in x.split('\n'):
            dates += parse_date(date, year)
        return dates
    return find_dates(x, year)


# print(parse_date('12.02–30.04 (12)', CURRENT_YEAR))
# print(parse_date('02.09–28.10 (9)\n11.11–09.12 (5)', CURRENT_YEAR))
# print(parse_date('01.09\n15.09\n29.09\n13.10\n27.10\n10.11', CURRENT_YEAR))
# print(parse_date('01.09\n15.09\n29.09\n27.10\n10.11', CURRENT_YEAR))

In [260]:
import datetime as dt
d1 = dt.datetime.strptime("2021-09-01", "%Y-%m-%d").date()
d2 = dt.datetime.strptime("2021-11-10", "%Y-%m-%d").date()
# ((d2-d1)/14).days
d1+dt.timedelta(days=10)

datetime.date(2021, 9, 11)

In [595]:
df_parsed_data = df['Date'].apply(lambda x: parse_date(x, CURRENT_YEAR))
df_parsed_data.name='Dates'

In [596]:
df = pd.concat([df, df_parsed_data], axis=1)

In [599]:
df_new = df.groupby(['Date', 'SubjectName', 'Time'])['Dates'].apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_3', axis = 1)
df_new.columns = ['Date', 'SubjectName', 'Time', 'DateStart','DateEnd', 'RepeatTime', 'RepeatType']
df_new

,Date,SubjectName,Time,DateStart,DateEnd,RepeatTime,RepeatType
0,01.09\n15.09\n29.09\n13.10\n27.10\n10.11,Электив. Компьютерное моделирование экономичес...,16:45–18:15,2021-09-01,2021-11-10,6,biweekly
1,01.09\n15.09\n29.09\n13.10\n27.10\n10.11,"Электив. Правовые информационные системы, прак...",16:45–18:15,2021-09-01,2021-11-10,6,biweekly
2,01.09\n15.09\n29.09\n13.10\n27.10\n10.11\n24.11,Электив. Инструментальные средства в решении о...,13:15–14:45,2021-09-01,2021-11-24,7,biweekly
3,01.09\n15.09\n29.09\n13.10\n27.10\n10.11\n24.11,Электив. Интеллектуальные информационные систе...,13:15–14:45,2021-09-01,2021-11-24,7,biweekly
4,01.09\n15.09\n29.09\n27.10\n10.11,Электив. Компьютерное моделирование экономичес...,15:00–16:30,2021-09-01,2021-09-29,3,biweekly
...,...,...,...,...,...,...,...
83,15.09–01.12 (12),"Операционные системы, среды и оболочки, практи...",10:45–12:15,2021-09-15,2021-12-01,12,weekly
84,18.12,"Теория игр, контрольная работа",13:15–14:45,2021-12-18,2021-12-18,1,weekly
85,22.09\n06.10\n20.10\n03.11\n17.11\n01.12,"Операционные системы, среды и оболочки, лекция",09:00–10:30,2021-09-22,2021-12-01,6,biweekly
86,27.09–29.11 (10),"Финансовая экономика, практическое занятие",15:00–16:30,2021-09-27,2021-11-29,10,weekly


In [600]:
df = pd.merge(df, df_new, on=['Date', 'SubjectName', 'Time'])
df = df.drop('Dates', axis=1)

In [577]:
df[df['SubjectName'].str.startswith('Траектория')]

,Time,Date,SubjectName,Place,Teacher,DateStart,DateEnd,RepeatTime
0,12:45–14:15,06.09–13.12 (15),"Траектория 1 (0 – В2). Английский язык, практи...",С использованием информационно-коммуникационны...,Петрова Е. Е.,2021-09-06,2021-12-13,15
1,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Быстрова Е. В.,2021-09-06,2021-12-13,15
2,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Залуцкая Н. О.,2021-09-06,2021-12-13,15
3,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Домшенко Н. Г.,2021-09-06,2021-12-13,15
4,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Солнцева Е. С.,2021-09-06,2021-12-13,15
5,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Вольфберг Д. М.,2021-09-06,2021-12-13,15
6,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Шарова В. В.,2021-09-06,2021-12-13,15
10,14:30–16:00,06.09–13.12 (15),"Траектория 1 (0 – В2). Английский язык, практи...",С использованием информационно-коммуникационны...,Петрова Е. Е.,2021-09-06,2021-12-13,15
11,14:30–16:00,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Быстрова Е. В.,2021-09-06,2021-12-13,15
12,14:30–16:00,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Солнцева Е. С.,2021-09-06,2021-12-13,15


In [522]:
def find_sub_group(subject: str):
    separation='_x000d_\n'
    if subject.find(separation) != -1:
        subject_name, sub_group = subject.split(separation)
    else:
        subject_name, sub_group= subject, ''
    return sub_group
# find_sub_group('Траектория 3 (В1 – В2). Английский язык, практическое занятие_x000d_\nПодгруппа 7')

In [601]:
df['SubGroup'] = df['SubjectName'].apply(lambda x: find_sub_group(x))

In [602]:
def parse_time(time_str):
    def get_datetime_format(time_str:str):
        return pd.to_datetime(time_str, format='%H:%M').time()
    return list(map(get_datetime_format, time_str.split('–')))

In [603]:
df_parsed_time = pd.DataFrame(df.groupby(['Time']).groups.keys())
df_parsed_time.columns = ['Time']

In [604]:
df_parsed_time = pd.concat([df_parsed_time, df_parsed_time.apply(lambda x: pd.Series(parse_time(x['Time'])), axis=1)], axis=1)
df_parsed_time.columns = ['Time', 'TimeStart', 'TimeEnd']

In [605]:
df = pd.merge(df, df_parsed_time, on=['Time'], how='left')

In [583]:
from icalendar import Calendar, Event
import datetime as dt
import pytz

In [607]:
class Subject:
    params = ['TimeStart', 'TimeEnd', 'DateStart', 'DateEnd', 'SubjectName', 'Place', 'Teacher', 'RepeatTime', 'RepeatType', 'SubGroup']
    tz = pytz.timezone('Europe/Moscow')

    def __init__(self, TimeStart, TimeEnd, DateStart, DateEnd, SubjectName, Place, Teacher, RepeatTime=1, RepeatType='weekly', SubGroup=None):
        self.time_start = TimeStart
        self.time_end = TimeEnd
        self.date_start = DateStart
        self.date_end = DateEnd
        self.subject_name = SubjectName if not SubGroup else SubjectName + '#' + f'{SubGroup}'
        self.place = Place
        self.teacher = Teacher
        self.repeat_time = RepeatTime
        self.repeat_type = RepeatType
        self.sub_group = SubGroup

    # @property
    # def time_start(self):
    #     return self._time_start
    #
    # @time_start.setter
    # def time_start(self, v):
    #     if type(v) == 'string':
    #         v =
    #     self._time_start = v
    @property
    def date_time_start_event(self):
        return self.get_date_time(self.date_start, self.time_start)

    @property
    def date_time_end_event(self):
        return self.get_date_time(self.date_start, self.time_end)

    @classmethod
    def get_date_time(cls, date, time):
        return dt.datetime.combine(date, time, cls.tz)

    @classmethod
    def from_series(cls, series: pd.Series):
        subject_params = [series[param] for param in cls.params]
        return cls(*subject_params)

    def __repr__(self):
        return f"<Subject time start: {self.time_start} date start: {self.date_start} with {self.repeat_time} repeat>"

    def to_event(self) -> Event:
        e = Event()
        e.add('DTSTART', self.date_time_start_event)
        e.add('DTEND', self.date_time_end_event)
        e.add('SUMMARY', self.subject_name)
        e.add('DESCRIPTION', f"Преподователь: {self.teacher}")
        e.add('LOCATION', self.place)
        e.add('LAST-MODIFIED', dt.datetime.now(self.tz))
        e.add('RRULE', {'freq': 'weekly', 'count': self.repeat_time, 'interval': 1 if self.repeat_type=='weekly' else 2})
        return e

In [608]:
cal = Calendar()
cal.add('prodid', '-//Google Inc//Google Calendar 70.9054//EN')

In [609]:
df = df[df['SubjectName'].str.match('((^(?!Траектория).*)(^(?!Русский язык).*))|(Траектория 3 .*)')]

In [606]:
# df.to_csv('autumn.csv')
df

,Time,Date,SubjectName,Place,Teacher,DateStart,DateEnd,RepeatTime,RepeatType,SubGroup,TimeStart,TimeEnd
0,12:45–14:15,06.09–13.12 (15),"Траектория 1 (0 – В2). Английский язык, практи...",С использованием информационно-коммуникационны...,Петрова Е. Е.,2021-09-06,2021-12-13,15,weekly,Подгруппа 1,12:45:00,14:15:00
1,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Быстрова Е. В.,2021-09-06,2021-12-13,15,weekly,Подгруппа 4,12:45:00,14:15:00
2,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Залуцкая Н. О.,2021-09-06,2021-12-13,15,weekly,Подгруппа 6,12:45:00,14:15:00
3,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Домшенко Н. Г.,2021-09-06,2021-12-13,15,weekly,Подгруппа 5,12:45:00,14:15:00
4,12:45–14:15,06.09–13.12 (15),"Траектория 2 (А2 – В2). Английский язык, практ...",С использованием информационно-коммуникационны...,Солнцева Е. С.,2021-09-06,2021-12-13,15,weekly,Подгруппа 2,12:45:00,14:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...
83,13:15–14:45,02.10–09.10 (2)\n23.10–30.10 (2),"Теория игр, лекция","Радищева улица, д. 39, лит. Д, пом. 1-Н,Р 11",Преподаватель,2021-10-23,2021-10-30,2,weekly,,13:15:00,14:45:00
84,13:15–14:45,06.11–11.12 (6),"Теория игр, практическое занятие","Радищева улица, д. 39, лит. Д, пом. 1-Н,Р 11",Преподаватель,2021-11-06,2021-12-11,6,weekly,,13:15:00,14:45:00
85,15:00–16:30,02.10–09.10 (2)\n23.10,"Теория игр, лекция","Радищева улица, д. 39, лит. Д, пом. 1-Н,Р 11",Преподаватель,2021-10-02,2021-10-09,2,weekly,,15:00:00,16:30:00
86,15:00–16:30,02.10–09.10 (2)\n23.10,"Теория игр, лекция","Радищева улица, д. 39, лит. Д, пом. 1-Н,Р 11",Преподаватель,2021-10-23,2021-10-23,1,weekly,,15:00:00,16:30:00


In [610]:
for idx, item in df.iterrows():
    s = Subject.from_series(item)
    e = s.to_event()
    cal.add_component(e)

In [611]:
with open('example_autumn.ics', 'wb') as f:
    data = cal.to_ical()
    f.write(data)

In [612]:
df

,Time,Date,SubjectName,Place,Teacher,DateStart,DateEnd,RepeatTime,RepeatType,SubGroup,TimeStart,TimeEnd
8,13:15–14:45,06.09–27.09 (4),"Финансовая экономика, лекция",С использованием информационно-коммуникационны...,Воронова Н. С.,2021-09-06,2021-09-27,4,weekly,,13:15:00,14:45:00
9,13:15–14:45,04.10–06.12 (10),"Финансовая экономика, практическое занятие",С использованием информационно-коммуникационны...,Воронова Н. С.,2021-10-04,2021-12-06,10,weekly,,13:15:00,14:45:00
18,15:00–16:30,06.12,"Финансовая экономика, контрольная работа",С использованием информационно-коммуникационны...,Воронова Н. С.,2021-12-06,2021-12-06,1,weekly,,15:00:00,16:30:00
19,15:00–16:30,06.09–20.09 (3),"Финансовая экономика, лекция",С использованием информационно-коммуникационны...,Воронова Н. С.,2021-09-06,2021-09-20,3,weekly,,15:00:00,16:30:00
20,15:00–16:30,27.09–29.11 (10),"Финансовая экономика, практическое занятие",С использованием информационно-коммуникационны...,Воронова Н. С.,2021-09-27,2021-11-29,10,weekly,,15:00:00,16:30:00
21,09:00–10:30,14.09\n28.09\n12.10\n26.10\n09.11\n23.11\n07.12,"Анализ и моделирование бизнес-процессов, практ...","Чайковского улица, д. 62, лит. А,302",Иванова В. В.,2021-09-14,2021-12-07,7,biweekly,,09:00:00,10:30:00
22,10:45–12:15,07.12,"Анализ и моделирование бизнес-процессов, контр...","Чайковского улица, д. 62, лит. А,302",Иванова В. В.,2021-12-07,2021-12-07,1,weekly,,10:45:00,12:15:00
23,10:45–12:15,07.09\n21.09\n05.10\n19.10\n02.11\n16.11\n30.11,"Анализ и моделирование бизнес-процессов, лекция","Чайковского улица, д. 62, лит. А,302",Иванова В. В.,2021-09-07,2021-11-30,7,biweekly,,10:45:00,12:15:00
24,10:45–12:15,14.09\n28.09\n12.10\n26.10\n09.11\n23.11,"Анализ и моделирование бизнес-процессов, практ...","Чайковского улица, д. 62, лит. А,302",Иванова В. В.,2021-09-14,2021-11-23,6,biweekly,,10:45:00,12:15:00
25,13:15–14:45,07.09–07.12 (14),Объектно-ориентированный анализ и программиров...,"Таврическая улица, д. 21-23-25, лит. А,36",Лебедев И. С.,2021-09-07,2021-12-07,14,weekly,,13:15:00,14:45:00


In [591]:
df[df['SubjectName'].str.startswith('Электив')].head()

,Time,Date,SubjectName,Place,Teacher,DateStart,DateEnd,RepeatTime,SubGroup,TimeStart,TimeEnd
34,13:15–14:45,01.12,Электив. Инструментальные средства в решении о...,С использованием информационно-коммуникационны...,Вьюненко Л. Ф.,2021-12-01,2021-12-01,1,,13:15:00,14:45:00
35,13:15–14:45,01.09\n15.09\n29.09\n13.10\n27.10\n10.11\n24.11,Электив. Инструментальные средства в решении о...,С использованием информационно-коммуникационны...,Вьюненко Л. Ф.,2021-09-01,2021-11-24,7,,13:15:00,14:45:00
36,13:15–14:45,08.09\n22.09\n06.10\n20.10\n03.11\n17.11,Электив. Инструментальные средства в решении о...,С использованием информационно-коммуникационны...,Вьюненко Л. Ф.,2021-09-08,2021-11-17,6,,13:15:00,14:45:00
37,13:15–14:45,01.12,Электив. Интеллектуальные информационные систе...,С использованием информационно-коммуникационны...,Войтенко С. С.,2021-12-01,2021-12-01,1,,13:15:00,14:45:00
38,13:15–14:45,01.09\n15.09\n29.09\n13.10\n27.10\n10.11\n24.11,Электив. Интеллектуальные информационные систе...,С использованием информационно-коммуникационны...,Войтенко С. С.,2021-09-01,2021-11-24,7,,13:15:00,14:45:00
